### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])



## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

## Calculate Requested Data

In [2]:
#Total number of schools
Total_Schools = school_data_complete["school_name"].value_counts()
Total_Schools.count()

15

In [3]:
#Total number of students
Total_Students = school_data_complete["student_name"].count()
Total_Students

39170

In [4]:
#Total budget
Total_budget = school_data_complete["budget"].unique()
Total_budget.sum()

24649428

In [5]:
#Math and reading average score
MathAverage = school_data_complete["math_score"].mean()
ReadingAverage = school_data_complete["reading_score"].mean()
#Overall Score
OverallAverageScore = round((MathAverage + ReadingAverage)/2,2)
OverallAverageScore


80.43

In [6]:
#Percentage of students with more than 70 score
PassMath = school_data_complete.loc[school_data_complete["math_score"] >= 70, "math_score"]
PassReading = school_data_complete.loc[school_data_complete["reading_score"] >= 70, "reading_score"]
PassMath.count(), PassReading.count()



(29370, 33610)

In [7]:
MathPassPercent = ["{0:.0%}".format(PassMath.count()/Total_Students)]
ReadingPassPercent = ["{0:.0%}".format(PassReading.count()/Total_Students)]
OverallPassPercent = ["{0:.0%}".format((PassMath.count()+PassReading.count())/(Total_Students*2))]
MathPassPercent , ReadingPassPercent, OverallPassPercent

(['75%'], ['86%'], ['80%'])

## Gathering data together

In [8]:
#Create a dateframe where to put results
District_Summary_Data = {"Total Schools": [Total_Schools.count()],
                         "Total Students" : Total_Students,
                         "Total Budget" : "${:0,.0f}".format(Total_budget.sum()),
                         "Math Average" : [round(MathAverage,2)],
                         "Readign Averag" : round(ReadingAverage,2),
                         "Overall Average" : round(OverallAverageScore,2),
                         "%Passing Math" : MathPassPercent,
                         "%Passing Reading" : ReadingPassPercent,
                         "%Overall Passing" : OverallPassPercent}
District_Summary = pd.DataFrame(District_Summary_Data)

#Deleting index from the result
blankIndex=[''] * len(District_Summary)
District_Summary.index=blankIndex
District_Summary

Total Schools  Total Students Total Budget  Math Average  Readign Averag  \
             15           39170  $24,649,428         78.99           81.88   

  Overall Average %Passing Math %Passing Reading %Overall Passing  
            80.43           75%              86%              80%

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Calculate requested data

In [9]:
#Getting the easiest of data 
School_Summary = school_data_complete.groupby(["school_name",'type' ,'budget']).agg(
    #I used the string format just to add the column budget without applying the function
                                                                Budget = (str('budget'), 'mean'),
                                                                Av_read = ("reading_score", "mean"),
                                                                Av_math = ("math_score", 'mean'),
                                                                Tot_stud = ('student_name', 'count'))
School_Summary.columns

Index(['Budget', 'Av_read', 'Av_math', 'Tot_stud'], dtype='object')

In [10]:
#Here I identifie the total students who passed by school in math and reading
PassMathperschool = school_data_complete.loc[school_data_complete["math_score"] >= 70, ["math_score","school_name",'type']]
PassReadingperschool = school_data_complete.loc[school_data_complete["reading_score"] >= 70,["reading_score","school_name",'type']]

PassMathperschool = PassMathperschool.groupby(['school_name','type']).count()
PassReadingperschool= PassReadingperschool.groupby(['school_name','type']).count()

PassMathperschool = pd.DataFrame(PassMathperschool)
PassReadingperschool = pd.DataFrame(PassReadingperschool)

PassMathperschool.head()

math_score
school_name          type                
Bailey High School   District        3318
Cabrera High School  Charter         1749
Figueroa High School District        1946
Ford High School     District        1871
Griffin High School  Charter         1371

In [11]:
#Merge data columns till now
alldata = pd.merge(School_Summary, PassReadingperschool, on= ['school_name','type'])
SchoolSummary = pd.merge(alldata, PassMathperschool, on= ['school_name','type'])

In [12]:
#Calculate percentages and add them in new columns, using the data of the new dataframe
SchoolSummary['Per Student Budget'] = (SchoolSummary['Budget']/SchoolSummary['Tot_stud'])
SchoolSummary['MathPercent'] = (SchoolSummary['math_score']/SchoolSummary['Tot_stud'])
SchoolSummary['ReadPercent'] = (SchoolSummary['reading_score']/SchoolSummary['Tot_stud'])
SchoolSummary['OverallPercent'] = (SchoolSummary['math_score']+SchoolSummary['reading_score'])/(SchoolSummary['Tot_stud']*2)
SchoolSummary.head()


Budget    Av_read    Av_math  Tot_stud  \
school_name          type                                                
Bailey High School   District  3124928  81.033963  77.048432      4976   
Cabrera High School  Charter   1081356  83.975780  83.061895      1858   
Figueroa High School District  1884411  81.158020  76.711767      2949   
Ford High School     District  1763916  80.746258  77.102592      2739   
Griffin High School  Charter    917500  83.816757  83.351499      1468   

                               reading_score  math_score  Per Student Budget  \
school_name          type                                                      
Bailey High School   District           4077        3318               628.0   
Cabrera High School  Charter            1803        1749               582.0   
Figueroa High School District           2381        1946               639.0   
Ford High School     District           2172        1871               644.0   
Griffin High School  Charter            1426        1371               625.0   

                               MathPercent  ReadPercent  OverallPercent  
school_name          type                                                
Bailey High School   District     0.666801     0.819333        0.743067  
Cabrera High School  Charter      0.941335     0.970398        0.955867  
Figueroa High School District     0.659885     0.807392        0.733639  
Ford High School     District     0.683096     0.792990        0.738043  
Griffin High School  Charter      0.933924     0.971390        0.952657

## Just Formatting! 

In [13]:
School_Summary = pd.DataFrame(SchoolSummary)
School_Summary.index.names = ['School Name', 'School Type']
School_Summary2 = School_Summary.rename(columns={'Av_read': 'Average Reading Score', 'Av_math' : 'Average Math Score', 
                                        'Tot_stud' : 'Total Students', 'reading_score' : 'ReadingPass',
                                        'math_score': 'MathPass', 'MathPercent' : "% Passing Math" ,
                                        'ReadPercent': "% Passing Reading ", 'OverallPercent' : '% Overall Passing Rate'})
School_Summary = School_Summary2.drop(['MathPass','ReadingPass'], axis=1)

School_Summary['Budget'] = ["${:0,.0f}".format(x) for x in School_Summary['Budget']]
School_Summary['Average Reading Score'] = [round(x,2) for x in School_Summary['Average Reading Score']]
School_Summary['Average Math Score'] = [round(x,2) for x in School_Summary['Average Math Score']]                                    
School_Summary['% Passing Math'] = ["{0:.0%}".format(x) for x in School_Summary['% Passing Math']]
School_Summary['% Passing Reading '] = ["{0:.0%}".format(x) for x in School_Summary['% Passing Reading ']]
School_Summary['% Overall Passing Rate'] = ["{0:.0%}".format(x) for x in School_Summary['% Overall Passing Rate']]                                  
        

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [14]:
School_Summary = School_Summary.sort_values(by=['% Overall Passing Rate'], ascending=False)
School_Summary.head()

Budget  Average Reading Score  \
School Name         School Type                                      
Cabrera High School Charter      $1,081,356                  83.98   
Griffin High School Charter        $917,500                  83.82   
Pena High School    Charter        $585,858                  84.04   
Shelton High School Charter      $1,056,600                  83.73   
Thomas High School  Charter      $1,043,130                  83.85   

                                 Average Math Score  Total Students  \
School Name         School Type                                       
Cabrera High School Charter                   83.06            1858   
Griffin High School Charter                   83.35            1468   
Pena High School    Charter                   83.84             962   
Shelton High School Charter                   83.36            1761   
Thomas High School  Charter                   83.42            1635   

                                 Per Student Budget % Passing Math  \
School Name         School Type                                      
Cabrera High School Charter                   582.0            94%   
Griffin High School Charter                   625.0            93%   
Pena High School    Charter                   609.0            95%   
Shelton High School Charter                   600.0            94%   
Thomas High School  Charter                   638.0            93%   

                                % Passing Reading  % Overall Passing Rate  
School Name         School Type                                            
Cabrera High School Charter                    97%                    96%  
Griffin High School Charter                    97%                    95%  
Pena High School    Charter                    96%                    95%  
Shelton High School Charter                    96%                    95%  
Thomas High School  Charter                    97%                    95%

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [15]:
School_Summary.tail()

Budget  Average Reading Score  \
School Name           School Type                                      
Hernandez High School District     $3,022,020                  80.93   
Huang High School     District     $1,910,635                  81.18   
Johnson High School   District     $3,094,650                  80.97   
Figueroa High School  District     $1,884,411                  81.16   
Rodriguez High School District     $2,547,363                  80.74   

                                   Average Math Score  Total Students  \
School Name           School Type                                       
Hernandez High School District                  77.29            4635   
Huang High School     District                  76.63            2917   
Johnson High School   District                  77.07            4761   
Figueroa High School  District                  76.71            2949   
Rodriguez High School District                  76.84            3999   

                                   Per Student Budget % Passing Math  \
School Name           School Type                                      
Hernandez High School District                  652.0            67%   
Huang High School     District                  655.0            66%   
Johnson High School   District                  650.0            66%   
Figueroa High School  District                  639.0            66%   
Rodriguez High School District                  637.0            66%   

                                  % Passing Reading  % Overall Passing Rate  
School Name           School Type                                            
Hernandez High School District                   81%                    74%  
Huang High School     District                   81%                    74%  
Johnson High School   District                   81%                    74%  
Figueroa High School  District                   81%                    73%  
Rodriguez High School District                   80%                    73%

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [16]:
nineth = school_data_complete.loc[school_data_complete["grade"] == "9th",['school_name', 'math_score']]
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th",['school_name', 'math_score']]
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th",['school_name', 'math_score']]
twelveth = school_data_complete.loc[school_data_complete["grade"] == "12th",['school_name', 'math_score']]

nineth = round(nineth.groupby(['school_name']).mean(),2)
tenth = round(tenth.groupby(['school_name']).mean(),2)
eleventh = round(eleventh.groupby(['school_name']).mean(),2)
twelveth = round(twelveth.groupby(['school_name']).mean(),2)

In [17]:
Math = pd.merge(nineth,tenth, on='school_name', suffixes=('9th', '10th'))
Math = pd.merge(Math, eleventh, on='school_name')
Math = pd.merge(Math, twelveth, on='school_name', suffixes=('11th','12th'))

Math = Math.rename(columns={'math_score9th' : '9th', 'math_score10th' : '10th', 'math_score11th' : '11th',
       'math_score12th': '12th'})
Math.index.name = 'School'
Math


9th   10th   11th   12th
School                                           
Bailey High School     77.08  77.00  77.52  76.49
Cabrera High School    83.09  83.15  82.77  83.28
Figueroa High School   76.40  76.54  76.88  77.15
Ford High School       77.36  77.67  76.92  76.18
Griffin High School    82.04  84.23  83.84  83.36
Hernandez High School  77.44  77.34  77.14  77.19
Holden High School     83.79  83.43  85.00  82.86
Huang High School      77.03  75.91  76.45  77.23
Johnson High School    77.19  76.69  77.49  76.86
Pena High School       83.63  83.37  84.33  84.12
Rodriguez High School  76.86  76.61  76.40  77.69
Shelton High School    83.42  82.92  83.38  83.78
Thomas High School     83.59  83.09  83.50  83.50
Wilson High School     83.09  83.72  83.20  83.04
Wright High School     83.26  84.01  83.84  83.64

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [18]:
nine = school_data_complete.loc[school_data_complete["grade"] == "9th",['school_name', 'reading_score']]
ten = school_data_complete.loc[school_data_complete["grade"] == "10th",['school_name', 'reading_score']]
eleven = school_data_complete.loc[school_data_complete["grade"] == "11th",['school_name', 'reading_score']]
twelve = school_data_complete.loc[school_data_complete["grade"] == "12th",['school_name', 'reading_score']]

nine = round(nine.groupby(['school_name']).mean(),2)
ten = round(ten.groupby(['school_name']).mean(),2)
eleven = round(eleven.groupby(['school_name']).mean())
twelve = round(twelve.groupby(['school_name']).mean())

In [19]:
Reading = pd.merge(nine,ten, on='school_name', suffixes=('9th', '10th'))
Reading = pd.merge(Reading, eleven, on='school_name')
Reading = pd.merge(Reading, twelve, on='school_name', suffixes=('11th','12th'))

Reading = Reading.rename(columns={'reading_score9th' : '9th', 'reading_score10th' : '10th', 'reading_score11th' : '11th',
       'reading_score12th': '12th'})
Reading.index.name = 'School'

layout : tuple ('Reading score')
Reading

9th   10th  11th  12th
School                                         
Bailey High School     81.30  80.91  81.0  81.0
Cabrera High School    83.68  84.25  84.0  84.0
Figueroa High School   81.20  81.41  81.0  81.0
Ford High School       80.63  81.26  80.0  81.0
Griffin High School    83.37  83.71  84.0  84.0
Hernandez High School  80.87  80.66  81.0  81.0
Holden High School     83.68  83.32  84.0  85.0
Huang High School      81.29  81.51  81.0  80.0
Johnson High School    81.26  80.77  81.0  81.0
Pena High School       83.81  83.61  84.0  85.0
Rodriguez High School  80.99  80.63  81.0  80.0
Shelton High School    84.12  83.44  84.0  83.0
Thomas High School     83.73  84.25  84.0  84.0
Wilson High School     83.94  84.02  84.0  84.0
Wright High School     83.83  83.81  84.0  84.0

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [20]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 598, 628, 648, 680]
group_names = ["<$600", "$600-630", "$630-650", ">$650"]

In [21]:
#I had to create a School_Summary2 because groupby mean doesn't applied to formated data
School_Summary2['Budget Category'] = pd.cut(School_Summary['Per Student Budget'], spending_bins, labels = group_names, include_lowest = True)
School_Summary2.groupby(['Budget Category'])['Average Reading Score','Average Math Score', '% Passing Math', '% Passing Reading ','% Overall Passing Rate'].mean()


Average Reading Score  Average Math Score  % Passing Math  \
Budget Category                                                              
<$600                        83.933814           83.455399        0.934601   
$600-630                     83.155286           81.899826        0.871335   
$630-650                     81.624473           78.518855        0.734842   
>$650                        81.027843           76.997210        0.661648   

                 % Passing Reading   % Overall Passing Rate  
Budget Category                                              
<$600                      0.966109                0.950355  
$600-630                   0.927182                0.899259  
$630-650                   0.843918                0.789380  
>$650                      0.811340                0.736494

## Scores by School Size

* Perform the same operations as above, based on school size.

In [22]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [23]:
School_Summary2['Size'] = pd.cut(School_Summary['Total Students'], size_bins, labels = size_names, include_lowest = True)
School_Summary2.groupby(['Size'])['Average Reading Score','Average Math Score', '% Passing Math', '% Passing Reading ','% Overall Passing Rate'].mean()


Average Reading Score  Average Math Score  % Passing Math  \
Size                                                                            
Small (<1000)                   83.929843           83.821598        0.935502   
Medium (1000-2000)              83.864438           83.374684        0.935997   
Large (2000-5000)               81.344493           77.746417        0.699634   

                    % Passing Reading   % Overall Passing Rate  
Size                                                            
Small (<1000)                 0.960994                0.948248  
Medium (1000-2000)            0.967907                0.951952  
Large (2000-5000)             0.827666                0.763650

## Scores by School Type

* Perform the same operations as above, based on school type.

In [35]:
#I make a mistake from the very beginnig, I worked with a multidex, until this point 
#that the request doesn't allow me to call Type because it was an index
School_Summary2.index = School_Summary2.index.droplevel(level=1)

IndexError: Too many levels: Index has only 1 level, not 2

In [59]:
#Here the index is fine now
School_Summary2.index

Index(['Bailey High School', 'Cabrera High School', 'Figueroa High School',
       'Ford High School', 'Griffin High School', 'Hernandez High School',
       'Holden High School', 'Huang High School', 'Johnson High School',
       'Pena High School', 'Rodriguez High School', 'Shelton High School',
       'Thomas High School', 'Wilson High School', 'Wright High School'],
      dtype='object', name='School Name')

In [69]:
#So i have to add again the type, but i couldnt
typeschool = school_data_complete[['school_name','type']]
types = typeschool.groupby(['school_name']).last()

In [70]:
School_Summary2['Type'] = types['type']

In [71]:
School_Summary2.groupby(['Type'])['Average Reading Score','Average Math Score', '% Passing Math', '% Passing Reading ','% Overall Passing Rate'].mean()

Average Reading Score  Average Math Score  % Passing Math  \
Type                                                                  
Charter               83.896421           83.473852        0.936208   
District              80.966636           76.956733        0.665485   

          % Passing Reading   % Overall Passing Rate  
Type                                                  
Charter             0.965865                0.951037  
District            0.807991                0.736738